In [45]:
import pandas as pd

data = pd.read_csv('../preprocess/202111171547_all.csv')
df = pd.DataFrame(data)
df = df.dropna()
data = df.iloc[:, :-1]
target = df['target']

In [46]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, stratify=target, random_state=0)
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, stratify=train_target, random_state=0)

In [47]:
from tensorflow import keras
import tensorflow.keras.backend as K

K.clear_session()
model = keras.Sequential()
model.add(keras.layers.SimpleRNN(32, activation="tanh", input_shape=(train_input.shape[1],1), return_sequences=True))
model.add(keras.layers.SimpleRNN(16, activation="tanh", dropout=0.3))
model.add(keras.layers.Dense(1, activation="relu"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 160, 32)           1088      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 16)                784       
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,889
Trainable params: 1,889
Non-trainable params: 0
_________________________________________________________________


In [48]:
model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

checkpoint_cb = keras.callbacks.ModelCheckpoint('11241516-best-rnn-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights=True)

history = model.fit(train_input, train_target, 
                    epochs=100, batch_size = 50,
                    validation_data = (val_input, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
64/64 [==============================] - 4s 40ms/step - loss: 3.6294 - accuracy: 0.5806 - val_loss: 0.7011 - val_accuracy: 0.6528
Epoch 2/100
64/64 [==============================] - 2s 38ms/step - loss: 1.9767 - accuracy: 0.5374 - val_loss: 0.6624 - val_accuracy: 0.6440
Epoch 3/100
64/64 [==============================] - 2s 36ms/step - loss: 1.1339 - accuracy: 0.5711 - val_loss: 0.6410 - val_accuracy: 0.6579
Epoch 4/100
64/64 [==============================] - 2s 38ms/step - loss: 0.7610 - accuracy: 0.6164 - val_loss: 0.5785 - val_accuracy: 0.8138
Epoch 5/100
64/64 [==============================] - 2s 35ms/step - loss: 0.6702 - accuracy: 0.7316 - val_loss: 0.5034 - val_accuracy: 0.7962
Epoch 6/100
64/64 [==============================] - 2s 34ms/step - loss: 0.6119 - accuracy: 0.7634 - val_loss: 0.4923 - val_accuracy: 0.7950
Epoch 7/100
64/64 [==============================] - 2s 34ms/step - loss: 0.5809 - accuracy: 0.7907 - val_loss: 0.5449 - val_accuracy: 0.8214
Epoch 

In [49]:
model.evaluate(val_input, val_target)

25/25 [==============================] - 0s 8ms/step - loss: 0.4923 - accuracy: 0.7950


[0.4923427700996399, 0.7949685454368591]

In [50]:
model.evaluate(test_input, test_target)

32/32 [==============================] - 0s 9ms/step - loss: 0.4705 - accuracy: 0.8119


[0.4705266058444977, 0.8118712306022644]